In [1]:
import pandas as pd 
import sqlite3
import pandasql

conn = sqlite3.connect("flights.db")
cur = conn.cursor

## **Aggregating Functions**

These are the functions that take in many values and return only one value.
These functions are like MIN(),MAX(),SUM(),AVG() etc

In [5]:
pd.read_sql("""
SELECT * FROM airports  
            
""",conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8107 entries, 0 to 8106
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      8107 non-null   int64 
 1   id         8107 non-null   object
 2   name       8107 non-null   object
 3   city       8107 non-null   object
 4   country    8107 non-null   object
 5   code       5880 non-null   object
 6   icao       8043 non-null   object
 7   latitude   8107 non-null   object
 8   longitude  8107 non-null   object
 9   altitude   8107 non-null   object
 10  offset     8107 non-null   object
 11  dst        8107 non-null   object
 12  timezone   8107 non-null   object
dtypes: int64(1), object(12)
memory usage: 823.5+ KB


In [7]:
#Examples of using Simple Aggregations

pd.read_sql("""
SELECT MAX(CAST(airports.longitude AS REAL))
FROM airports ;
            
""",conn)

,MAX(CAST(airports.longitude AS REAL))
0,179.951


In [10]:
#Using the count to count all the not active airlines

pd.read_sql("""
SELECT COUNT() AS no_of_inactive_airlines
FROM airlines
  WHERE 
      active = "N"
           
""",conn)

,no_of_inactive_airlines
0,4886


In [13]:
pd.read_sql("""
SELECT COUNT() AS no_of_active_airlines
FROM airlines
  WHERE 
      active = "Y"
           
""",conn)

,no_of_active_airlines
0,1161


## Grouping in SQL

In [16]:
# We can do the above task shich has 2 queries to check the active and inactive airlines

df_result = pd.read_sql("""
                    SELECT
                        airlines.active, 
                        COUNT() AS number_of_airlines
                    FROM airlines
                    GROUP BY airlines.active;
                     """,conn)
df_result

,active,number_of_airlines
0,N,4886
1,Y,1161
2,n,1


Which country has the highest number of active airlines?

In [18]:
pd.read_sql('''
            SELECT * FROM airlines
            
            ''',conn).head()

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [25]:
pd.read_sql("""
            SELECT airlines.country, COUNT() AS country_active_airlines
            FROM airlines
            WHERE airlines.active = "Y"
            GROUP BY airlines.country
            ORDER BY country_active_airlines DESC
            LIMIT 10  
            """,conn)

,country,country_active_airlines
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34
5,Australia,26
6,China,25
7,Spain,24
8,Brazil,23
9,France,22


Which country has the highest number of inactive airlines? Return all the countries that have more than 10


In [34]:
pd.read_sql("""
            SELECT COUNT() AS country_inactive_airlines,airlines.country
            FROM airlines
            WHERE airlines.active IN ("N","n") 
            GROUP BY airlines.country
            HAVING country_inactive_airlines > 10
            ORDER BY country_inactive_airlines DESC
            
            """,conn)

,country_inactive_airlines,country
0,939,United States
1,427,Mexico
2,367,United Kingdom
3,284,Canada
4,158,Russia
...,...,...
68,12,Georgia
69,12,Cambodia
70,11,Luxembourg
71,11,Iceland


Run a query that will return the number of airports by time zone. Each row should have a number of airports and a time zone.

In [37]:
pd.read_sql("""
            SELECT * FROM airports
            """,conn).head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [35]:
pd.read_sql("""
            SELECT airports.timezone,COUNT() AS num_of_airports
            FROM airports
            
            GROUP BY airports.timezone
            ORDER BY num_of_airports DESC
            
            """,conn)

,timezone,num_of_airports
0,America/New_York,628
1,America/Chicago,373
2,Europe/Berlin,319
3,America/Anchorage,258
4,Europe/Paris,232
...,...,...
289,America/Anguilla,1
290,Africa/Porto-Novo,1
291,Africa/Mbabane,1
292,Africa/Bujumbura,1


The `HAVING` clause should be used after the `GROUP BY`

## Filtering Different Aggregation -Airport Altitudes

We can also filter on other aggregations. For example, let's say we want to investigate the airports table.

Specifically, we want to know the height of the highest airport in a country given that it has at least 100 airports.

In [39]:
#cheking the airports table

pd.read_sql("""SELECT * FROM airports""",conn).head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [44]:
pd.read_sql("""
            SELECT  airports.country,MAX(CAST(airports.altitude AS REAL)) AS highest_airport_in_country , COUNT() AS number_of_airports_in_country
            
            FROM airports
            GROUP BY airports.country
            HAVING count() > 100
            ORDER BY airports.country DESC
            
            """,conn)

,country,highest_airport_in_country,number_of_airports_in_country
0,United States,9308.0,1697
1,United Kingdom,811.0,210
2,South Africa,5700.0,103
3,Russia,13411.0,249
4,Japan,2200.0,131
5,Indonesia,9104.0,114
6,India,10682.0,140
7,Germany,2297.0,321
8,France,6588.0,233
9,China,14472.0,219


## Joins

In [45]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes 
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


In [46]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,index,id,name,alias,iata,icao,callsign,country,active
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66980,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,4171,4178,Regional Express,\N,ZL,RXA,REX,Australia,Y
66981,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66982,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66983,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y


we can specify to retain the certain columns in SELECT clause

In [47]:
pd.read_sql('''
    SELECT 
        routes.source AS departing
        ,routes.dest AS destination
        ,routes.stops AS stops_before_destination
        ,airlines.name AS airline
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

,departing,destination,stops_before_destination,airline
0,AER,KZN,0,Aerocondor
1,ASF,KZN,0,Aerocondor
2,ASF,MRV,0,Aerocondor
3,CEK,KZN,0,Aerocondor
4,CEK,OVB,0,Aerocondor
...,...,...,...,...
66980,WYA,ADL,0,Regional Express
66981,DME,FRU,0,Apache Air
66982,FRU,DME,0,Apache Air
66983,FRU,OSS,0,Apache Air


Check the data lost due to the inner joins

In [48]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

df_routes_after_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

In [49]:
# Look at how the number of rows are different
df_all_routes.shape, df_routes_after_join.shape 

((67663, 10), (66985, 19))

In [50]:
# This will include all the data from routes
df_routes_after_left_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        LEFT JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

df_routes_after_left_join.shape

(67663, 19)